Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [6]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [5]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES)

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 15400 | Peso: 12.1
Maior valor: 15500 | Peso: 13.5
Maior valor: 17900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Harry Potter: ele voltou, confia!
+ Quadrinho super raro do Aranha-Homem da vida real
+ energético
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 17900 dinheiros e peso de 14.1 unidades de massa.


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.

## Discussão 

Para abstrair o conceito imaterial de mochilas para programação vamos usar um dicionário, onde cada chave é um nome e cada um contem as informações de peso e valor. Nosso objetivo é maximizar o valor da soma dos valores dos objetos dentro da mochila e mantar o limite máximo de peso.

O "indivíduo" desse problema vão ser compostos pelos itens da mochila, não necessáriamente os seus pesos e valores, portanto precisamos "voltar" as informações do que cada item contem. Vamos fazer uma lista de 0, e cada posição representa um item, sempre na mesma ordem, se o item existe na mochila então trocamos 0 por 1. Maximizamos o valor de cada mochila ao somar os valores dentro da lista de zeros e uns. Igual ao problema das caixas binárias.

A função computa mochila recebe um indivíduo, objetos disponíveis e a ordem que eles estão dispostos, então resgatamos os valores e pesos usando o ordem que eles estão e a posição dessas informações (valor e peso), para então ser computado.

Na função objetivo vamor usar um valor baixo como punição, assim diminui a chance de uma mochila com o peso maior que o limite de continuar nas próximas gerações.
A seleção vai pegar os indivíduos com o melhor fitness

## Conclusão

O problema da mochila envolve encontrar um conjunto de itens de um número que definimos em nosso dicionário, cada um com um peso e valor associado, que maximize o valor total contido na mochila, considerando a capacidade limitada da mesma. A capacidade é definida como um número que a mochila pode suportar sem se rasgar, também definimos essa variável. A solução deve garantir que o peso total dos itens selecionados não ultrapasse a capacidade da mochila. O objetivo é maximizar o valor total dos itens, enquanto se mantém dentro da capacidade de peso da mochila.

Diferente de outros experimentos, dessa vez nós demos as opções de "genes" que podem estar dentro do indíviduo, e não pedimos para ele gerar aleatóriamente, apenas pedimos para escolher quais que vão dentro do indivíduo. Essa é um outro desafio do qual não sabemos o resultato otimizadp, portanto, usamos o número de gerações como critério de parada para nossos agoritmos genéticos. 

## Playground

